In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv('/content/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')

In [ ]:
data.head(1)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.isnull().any()

essay_id          False
essay_set         False
essay             False
rater1_domain1    False
rater2_domain1    False
rater3_domain1     True
domain1_score     False
rater1_domain2     True
rater2_domain2     True
domain2_score      True
rater1_trait1      True
rater1_trait2      True
rater1_trait3      True
rater1_trait4      True
rater1_trait5      True
rater1_trait6      True
rater2_trait1      True
rater2_trait2      True
rater2_trait3      True
rater2_trait4      True
rater2_trait5      True
rater2_trait6      True
rater3_trait1      True
rater3_trait2      True
rater3_trait3      True
rater3_trait4      True
rater3_trait5      True
rater3_trait6      True
dtype: bool

In [ ]:
data = data.dropna(axis=1)
data = data.drop(columns=['rater1_domain1','rater2_domain1'])

In [ ]:
data.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [ ]:
x=data.iloc[:,0:3]
y=data.iloc[:,3:4]

In [ ]:
x

,essay_id,essay_set,essay
0,1,1,"Dear local newspaper, I think effects computer..."
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu..."
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl..."
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that..."
4,5,1,"Dear @LOCATION1, I know having computers has a..."
...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...
12972,21628,8,I never understood the meaning laughter is th...
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ..."
12974,21630,8,Trippin' on fen...


In [ ]:
y

,domain1_score
0,8
1,9
2,7
3,10
4,8
...,...
12971,35
12972,32
12973,40
12974,40


In [ ]:
minsrc = []
maxsrc = []
for i in range(1,9):
  minsrc.append(data[data['essay_set']==i]['domain1_score'].min())
  maxsrc.append(data[data['essay_set']==i]['domain1_score'].max())
print("minimum score= ",minsrc )
print("maximum score= ",maxsrc )

minimum score=  [2, 1, 0, 0, 0, 0, 2, 10]
maximum score=  [12, 6, 3, 3, 4, 4, 24, 60]


In [ ]:
def essay_to_wordlist(essay_v):
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    return (words)

#Tokenize the senstences and call essay_to_wordlist() for word tokenization.
def essay_to_sentences(essay_v):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence))
    return sentences

In [ ]:
#Feature vector is made from the words list of an essay.
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2entity)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

#Word vectors are generated for Word2Vec model
def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

In [ ]:
x_train.shape

(10380, 3)

In [ ]:
x_test.shape

(2596, 3)

In [ ]:
train_essays = x_train['essay']
test_essays = x_test['essay']

In [ ]:
sentences = []
# Obtaining all sentences from the training essays.
for essay in train_essays:
    sentences += essay_to_sentences(essay)

In [ ]:
sentences[0]

['essay', 'author', 'talking', 'journey']

In [ ]:
model = Word2Vec(sentences, workers = 4, size= 300, min_count= 40, window = 10, sample = 1e-3)

In [ ]:
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

In [ ]:
clean_train_essays = []
for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, 300)
 
#get the testing vectors
clean_test_essays = []
for essay_v in test_essays:
    clean_test_essays.append(essay_to_wordlist( essay_v))
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, 300 )
    
#convert the vectors to numpy array
trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

In [ ]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

In [ ]:
model = Sequential()

In [ ]:
model = Sequential()
model.add(LSTM(300,dropout=0.4, recurrent_dropout=0.4, input_shape=[1,300], return_sequences=True))
model.add(LSTM(64, recurrent_dropout=0.4))
model.add(Dropout(0.5))
model.add(Dense(1,activation='relu'))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])

In [ ]:
model.fit(trainDataVecs, y_train, batch_size=64, epochs = 100)

Epoch 1/100
163/163 [==============================] - 8s 25ms/step - loss: 51.8935 - mae: 3.6380
Epoch 2/100
163/163 [==============================] - 4s 25ms/step - loss: 28.0224 - mae: 2.4557
Epoch 3/100
163/163 [==============================] - 4s 25ms/step - loss: 18.4797 - mae: 2.1077
Epoch 4/100
163/163 [==============================] - 4s 25ms/step - loss: 12.6890 - mae: 1.8561
Epoch 5/100
163/163 [==============================] - 4s 25ms/step - loss: 10.1816 - mae: 1.7022
Epoch 6/100
163/163 [==============================] - 4s 25ms/step - loss: 9.1130 - mae: 1.6300
Epoch 7/100
163/163 [==============================] - 4s 25ms/step - loss: 8.4435 - mae: 1.5657
Epoch 8/100
163/163 [==============================] - 4s 25ms/step - loss: 7.8569 - mae: 1.5142
Epoch 9/100
163/163 [==============================] - 4s 25ms/step - loss: 7.7349 - mae: 1.5185
Epoch 10/100
163/163 [==============================] - 4s 25ms/step - loss: 7.5353 - mae: 1.4778
Epoch 11/100
163/163 [==

In [ ]:
testDataVecs.shape

(2596, 1, 300)

In [ ]:
y_train.shape

(10380, 1)

In [ ]:
ypred = model.predict(testDataVecs)

In [ ]:
ypred

array([[12.615318 ],
       [15.47917  ],
       [ 3.1516871],
       ...,
       [ 1.917428 ],
       [ 2.8116941],
       [ 2.8596215]], dtype=float32)

In [ ]:
y_test

,domain1_score
10823,15
11018,17
2263,4
11074,20
12491,33
...,...
6906,2
8418,3
5226,2
8992,3


In [ ]:
model.save('final_lstm.h5')

In [ ]:
from sklearn.metrics import r2_score
accuracy = r2_score(y_test,ypred)
accuracy

0.939725818586611

In [ ]:

import gensim.models.keyedvectors as word2vec

In [ ]:
testsen='''Dear local newspaper, I\'ve heard that not many people think computers benefit society. I disagree with that. Computers benefit society by teaching hand-eye coordination, allowing people to learn about foregin places, and allow people to communicate with others online. Some people were basically born with hand-eye coordination, but most people weren\'t, like me, for example. But, thanks to computers, my hand-eye coordination has gone up! "On average, computers can increase a persons hand-eye coordination by up to @PERCENT1 says @PERSON2. Think of it, just by learning to tye, your hand-eye coordination can go up @PERCENT1.! That\'s amazing! Having good hand-eye coordination will get you far in life, and helps with small, everyday things such as playing an instrument, or doing household chores. It also helps in sports. You need good hand-eye coordination to throw, catch, and aim. If more people spend more time on the computer, then the population\'s hand-eye coordination would increase, and who wouldn\'t want thats. Have you ever heard of @LOCATION1? It is a country in @LOCATION2 that no one really knows much about. By looking it up on the computer, you can become an expert on the country! The computer gives us access to a lot of knowledge, and will make us more of people around the world. This can improve our travel. Instead of going to the same place every vacation, you can go to a place you saw online that looks just as exciting. Also, by learning about other countries and the people in them, we gain respect for them. We @MONTH1 also learn about their traditions, and culture. "The internet gives us access to the world," says @PERSON1, "we can learn so many new things that will make us more of people, countries, and the environment." @CAPS1 is right, we can also be aware of the environment. Since global warming is a problem, we can learn how o preserve foreign places by learning about them on the internet. With a click of a mouse you can virtually have the whose world at your fingertips! In a recent study @PERCENT2 of people said they don\'t have time to meet with old friends. The internet, however has made it quick and easy to talk to people online. Websites like @CAPS2, @CAPS3, and @CAPS4 can keep your social life in tact, and can even reunite yu with old friends. For example, @CAPS2 suggest family. You can keep in touch wih many people trough @CAPS2! Also, you can make new friends. Most teens say they are always willing to meet new people, and the internet is a great way to do so! Sites like @CAPS7, and @CAPS8 can can help you find friends by connecting you in a chat room to a random person, just to talk. I once talked with someone on @CAPS7 for over an hour! We found that we both have @CAPS4 accounts and keep in touch there! This also helps you gain confidence because you @MONTH1 have the courage to say something over the computer that you would\'t say real life. The computer can really amp up your social life. As you can see. computers don\'t just "your barain," they imprdove hand-eye coordination, help you learn about far away places, and improve your social life by allowing you to talk to others. Computers benefit the society more than anymore will ever know!'''

In [ ]:
model1 = word2vec.KeyedVectors.load_word2vec_format('word2vecmodel.bin', binary=True)
index2word_set = set(model1.index2entity)

In [ ]:
testsen2 = re.sub("[^a-zA-Z]", " ", testsen)
testsen2 = essay_v.lower()
featureVec = np.zeros((300,),dtype="float32")
for word in testsen2:
        if word in index2word_set:
            featureVec = np.add(featureVec,model1[word])

In [ ]:
featureVec.shape
avc=featureVec.reshape(1,1,300)

In [ ]:
from keras.models  import load_model
model2 = load_model("final_lstm.h5")

In [ ]:
y_pred = model2.predict(avc)

In [ ]:
y_pred

array([[4.0548315]], dtype=float32)